In [2]:
import pandas as pd
import numpy as np
# import nltk
# nltk.download('punkt')
from collections import Counter
# import fasttext.util
import os
import re
from collections import Counter
from tqdm.auto import tqdm
tqdm.pandas()
# fasttext.util.download_model('en', if_exists='ignore')

# HuggingFace

In [7]:
# ! pip install transformers

In [8]:
#  ! pip install datasets

In [11]:
import pandas as pd
import numpy as np

import torch

from transformers import AutoTokenizer
from datasets import load_dataset

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

In [10]:
torch.cuda.is_available()

False

## Preparing the data

In [25]:
# doc_df = pd.read_csv('data/Thoughts_en.doc.tsv', sep='\t')
doc_df = pd.read_csv('data/subs.actions.tsv', sep='\t')

In [26]:
doc_df = doc_df.dropna()

In [27]:
msk = np.random.rand(len(doc_df)) < .95

In [28]:
train_doc_df = doc_df[msk]

In [29]:
test_doc_df = doc_df[~msk]

In [30]:
# train_doc_df.to_csv('data/thoughts/Thoughts_en.doc.train.tsv', header=['text','label'], index=False, sep='\t')
train_doc_df.to_csv('data/subs.actions.train.tsv', header=['text','label'], index=False, sep='\t')

In [31]:
# test_doc_df.to_csv('data/thoughts/Thoughts_en.doc.test.tsv', header=['text','label'], index=False, sep='\t')
test_doc_df.to_csv('data/subs.actions.test.tsv', header=['text','label'], index=False, sep='\t')

In [45]:
# vec_df = pd.read_csv('data/thoughts/Thoughts_en.vec.tsv', sep='\t', header=None)

In [46]:
# train_vec_df = vec_df[msk]

In [203]:
# test_vec_df = vec_df[~msk]

In [209]:
# train_vec_df.to_csv('data/Thoughts_en.vec.train.tsv', index=False, header=False, sep='\t')

In [210]:
# test_vec_df.to_csv('data/Thoughts_en.vec.test.tsv', index=False, header=False, sep='\t')

In [222]:
# train_vec_df['label'] = train_doc_df['is_action']

In [223]:
# test_vec_df['label'] = test_doc_df['is_action']

## Training

In [224]:
# train_vec_df.to_csv('data/Thoughts_en.vec.train.labeled.tsv', index=False, sep='\t')
# test_vec_df.to_csv('data/Thoughts_en.vec.test.labeled.tsv', index=False, sep='\t')

In [32]:
# dataset = load_dataset('csv', data_files={'train': 'data/thoughts/Thoughts_en.doc.train.tsv',
#                                               'test': 'data/thoughts/Thoughts_en.doc.test.tsv'}, delimiter='\t')
dataset = load_dataset('csv', data_files={'train': 'data/subs.actions.train.tsv',
                                              'test': 'data/subs.actions.test.tsv'}, delimiter='\t')

Using custom data configuration default-abee3eae27b5665e


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-abee3eae27b5665e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [63]:
# dataset = load_dataset("imdb")

In [33]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 176727
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9305
    })
})

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [35]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=20)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/177 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [36]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [37]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [41]:
training_args = TrainingArguments("subs_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
# trainer = Trainer(
#     model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
# )
trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

In [13]:
trainer.train()

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("subs_trainer/checkpoint-66000/")

In [65]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt")
    labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
    outputs = model(**inputs, labels=labels)
    return torch.nn.Softmax(dim=1)(outputs.logits).tolist()[0][1]
    

In [66]:
predict("Alice, you need to finish my cereal by tomorrow")
# inputs = tokenizer("Fuck you", return_tensors="pt")



0.9999179840087891

In [67]:
subs_df = pd.read_csv('data/subs.actions.test.tsv', sep='\t')

In [69]:
subs_df = subs_df[subs_df['label']==False]

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [71]:
subs_df['prediction'] = subs_df['text'].progress_apply(predict)

  0%|          | 0/4636 [00:00<?, ?it/s]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
subs_df = subs_df.sort_values('prediction', ascending=False)

In [76]:
subs_df.iloc[1,0]

'Downloading the data would cause death to the data carrier.'

## Use POS to identify Action Items

In [5]:
# ! pip install nltk

In [6]:
# ! python -m spacy download en

In [7]:
import spacy
from string import punctuation
import nltk
import nltk.data
from collections import Counter
from functools import partial

In [16]:
# nltk.download()

In [8]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [9]:
nlp = spacy.load("en_core_web_sm")

In [19]:
doc_df = pd.read_csv('data/thoughts/Thoughts_en.doc.tsv', sep='\t')

In [20]:
doc_df = pd.read_csv('data/subs.actions.tsv', sep='\t')

In [21]:
doc_df = doc_df[doc_df['label']==False]

In [10]:
doc_df = pd.read_csv('data/ami.sentence.tsv', sep='\t')

In [11]:
# verbs = []
def is_action(text):
    if type(text)!=str:
        return False
    if "?" in text:
        return False
#     if 'need' in text:
#         return True
    text = re.sub(',', '.', text)
#     split = tokenizer.tokenize(text)
    sents = nlp(text)
    for sent in sents.sents:
        if not sent:
            continue
        for word in sent:
            if word.dep_=='neg':
                return False
            if word.dep_=='ROOT' and word.pos_=='VERB':
                try:
                    morph = word.morph.to_dict()
                    if morph['VerbForm']=='Inf':
                        return True
                except KeyError:
                    continue
#         if sen[0].pos_=='VERB':
#             return True
#     #     if '!' in x and any(word.pos_=='VERB' for word in sen): # Doesn't work
#     #         return True
    return False

In [12]:
# doc_df['is_action'] = doc_df['doc'].progress_apply(is_action)
doc_df['is_action'] = doc_df['text'].progress_apply(is_action)

  0%|          | 0/83436 [00:00<?, ?it/s]

In [18]:
pd.Series({'names':['hello'], 'locations':['paris']})

names        [hello]
locations    [paris]
dtype: object

In [85]:
def parse_sent(sent):
    parse = {'dates':[], 'locations':[], 'names':[]}
    try:
        sentence = nlp(sent)
    except ValueError:
        return pd.Series(parse)
    ents = sentence.ents
    for token in sentence:
        if token.ent_type_ == "DATE":
            parse['dates'] += [token]
        if token.ent_type_ == "GPE":
            parse['locations'] += [token]
        if token.pos_!='PROPN':
            continue
        if token.ent_type_ in ["DATE", "GPE"]:
            continue
        parse['names']+= [token]
    return pd.Series(parse)

In [93]:
parse_df = doc_df.loc[:, 'text'].progress_apply(parse_sent)

  0%|          | 0/83436 [00:00<?, ?it/s]

In [100]:
doc_df[['dates','locations','names']] = parse_df

In [105]:
# doc_df= doc_df.drop('parse', axis=1)

In [89]:
def get_names(sent):
    try:
        sentence = nlp(sent)
    except ValueError:
        return []
    names = []
    ents = sentence.ents
    for token in sentence:
        if token.pos_!='PROPN':
            continue
        if token.ent_type_ in ["DATE", "GPE"]:
            continue
        names.append(token)
    return names

In [94]:
# doc_df['names'] = doc_df['text'].progress_apply(get_names)

In [ ]:
def get_ents(sent, type_):
    try:
        sentence = nlp(sent)
    except ValueError:
        return []
    ents = []
    for entity in sentence.ents:
        if entity.label_==type_:
            ents.append(entity)
    return ents
    

In [ ]:
get_dates = partial(get_ents, type_="DATE")

In [ ]:
get_locations = partial(get_ents, type_="GPE")

In [ ]:
doc_df['dates'] = doc_df['text'].progress_apply(get_dates)

In [ ]:
doc_df['locations'] = doc_df['text'].progress_apply(get_locations)

In [47]:
# doc_df.to_csv('data/Thoughts_en.doc.tsv', index=False, sep='\t')

In [ ]:
# doc_df.to_csv('data/subs.action.name.date.doc.tsv', index=False, sep='\t')

In [106]:
doc_df.to_csv('data/ami.sentence.labelled.tsv', index=False, sep='\t')

In [107]:
actions_df = doc_df[ doc_df['is_action'] ]

In [148]:
# actions_df.to_csv('data/subs.action.name.date.doc.tsv', index=False, sep='\t')

In [108]:
actions_df.to_csv('data/ami.action.tsv', index=False, sep='\t')

In [109]:
actions_df

,starttime,text,fname,is_action,dates,locations,names
0,64.47,Well I'll uh start just with another presentat...,TS3009b.A.words.xml,True,[],[],[]
6,99.75,Well um in we'll uh just have a look at the at...,TS3009b.A.words.xml,True,[],[],[dec]
7,110.82,Uh then we'll uh look at uh the three uh prese...,TS3009b.A.words.xml,True,[],[],[]
9,132.31,Oh I've received a mail with uh some additiona...,TS3009b.A.words.xml,True,[],[],[]
11,159.54,And then uh we can discuss uh some more closel...,TS3009b.A.words.xml,True,[],[],[]
...,...,...,...,...,...,...,...
83413,2301.58,"Break it , I don't get",TS3007b.B.words.xml,True,[],[],[]
83419,2406.55,"'Kay , save it as an image on the res",TS3007b.B.words.xml,True,[],[],[Kay]
83420,2420.00,"No , and use an image if possible .",TS3007b.B.words.xml,True,[],[],[]
83430,2471.98,"The questionnaire , fill in uh we fill out d a...",TS3007b.B.words.xml,True,[],[],[]


# Generate synthetic examples

In [3]:
import random

In [4]:
# time
# time template
preposition = ["by","for","until", 'till', '']
timeframe = ["end of", "next", "this", ""]
day = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "today", "tomorrow", "week", "the 20th", "the first of July", ""]
time = ["3pm", "afternoon", "morning", "at 5pm", "noon", ""]

def gen_time():
    prep = random.choice(preposition)
    tf = random.choice(timeframe)
    d = random.choice(day)
    t = random.choice(time)

    return " ".join([prep, tf, d, t])

# 
# name
# action


In [5]:
# ! pip install lxml

In [7]:
name_dfs = pd.read_html('https://www.ssa.gov/oact/babynames/decades/century.html')

In [8]:
name_df = name_dfs[0]

In [9]:
names = list(name_df['Males']["Name"])+list(name_df['Females']["Name"])

In [10]:
# action
asks = ["can you please", "can you", "please", "", "you need to"]
actions = ["read", "write", "fix", "go over", "correct", "submit", "discuss", "run", "execute", "improve", "purchase", "review"]
adjectives = ["the last", "the", "the new", "this", "that", "the old"]
subjects = ["report", "code", "account", "review", "data", "record", "description"]

def gen_action():
    a = random.choice(asks)
    act = random.choice(actions)
    adj = random.choice(adjectives)
    subj = random.choice(subjects)

    return " ".join([a, act, adj, subj])

In [11]:
templates = [
    "{name}, {action} {time}",
    "{name}, {time} {action}",
    "{action} {time}, {name}",
    "{time} {action}, {name}"
]

def gen_action_item():
    name = random.choice(names)
    action = gen_action()
    time = gen_time()
    temp = random.choice(templates)
    
    

    action_item = temp.format(name=name, action=action, time=time)
    return action_item

In [12]:
gen_action_item()

' end of Thursday morning you need to improve this data, Brandon'

# OpenSubtitles

In [13]:
open_sub = load_dataset('open_subtitles', 'en-hi')

Reusing dataset open_subtitles (/root/.cache/huggingface/datasets/open_subtitles/en-hi/2018.0.0/c1ec973ca4b6e588740d8f167cc0e24ea3f626e70bc7ffe467e944730500e198)


  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
en_sub = [x['en'] for x in open_sub['train']['translation']]

In [16]:
action_items = [gen_action_item() for _ in range(len(en_sub))]

In [21]:
subs_df = pd.DataFrame(list(zip(en_sub, [False]*len(en_sub))) + list(zip(action_items,[True]*len(action_items))), columns=['text', 'label'])

In [22]:
subs_df.head()

,text,label
0,THE BICYCLE THIEF,False
1,Ricci?,False
2,Is Ricci there?,False
3,Are you deaf? Come on!,False
4,Get a move on.,False


In [23]:
subs_df.tail()

,text,label
186027,please submit this description next the 20th ...,True
186028,you need to write the old review by next Tuesd...,True
186029,"Patrick, till this the 20th noon can you submi...",True
186030,can you go over the new account till this Frid...,True
186031,"Donna, until afternoon please fix the last r...",True


In [24]:
subs_df.to_csv('data/subs.actions.tsv', index=False, sep='\t')

In [ ]:
def is_action(text):
    sen = nlp(text)
    
    

In [66]:
# subs_df[subs_df['text'].apply(lambda x: '!' in x, sen = nlp(x))]

# Meeting minutes data

In [11]:
import xml.etree.ElementTree as ET

In [ ]:
# ! wget -P data/ http://groups.inf.ed.ac.uk/ami/AMICorpusAnnotations/ami_public_manual_1.6.2.zip

In [5]:
! mkdir data/ami_public_manual_1.6.2

In [7]:
# ! unzip data/ami_public_manual_1.6.2.zip -d data/ami_public_manual_1.6.2/

In [8]:
# ! wget -P data/ https://webtransfer.vancouver.ca/OpenData/txt/CouncilMeetingMinutesArchival.txt

In [ ]:
! head data/ami_public_manual_1.6.2/

In [111]:
def extract_sentences(fname):
    tree = ET.parse('data/ami_public_manual_1.6.2/words/'+fname)
    root = tree.getroot()
    data = []
    for node in root:
        try:
            data.append([node.text, node.attrib['starttime'], node.attrib['endtime']])
        except KeyError:
            continue

    if not data:
        return []
            
    # # checking the word distance histogram to decide how to identify new sentences
    # df = pd.DataFrame(data, columns=['text', 'starttime', 'endtime'])
    # df['starttime'] = pd.to_numeric(df['starttime'])
    # df['endtime'] = pd.to_numeric(df['endtime'])
    # df['length'] = df['endtime'] - df['starttime']
    # df['diff'] = df['endtime'].diff()
    # df['dist'] = df['diff']-df['length']
    # df['dist'].hist(bins=[1*i for i in range(10)])

    sentences = []
    prev,prev_s,prev_e = data[0]
    sentence = prev+" " if prev else ""
    sentence_s = prev_s
    for word,s,e in data[1:]:
        if not word:
            continue
        if float(s)-float(prev_e)>0.1:
            sentences.append((sentence_s,sentence,fname))
            sentence_s = s
            sentence = ""
        sentence += word+" "
        prev,prev_s,prev_e = word,s,e
    return sentences

In [118]:
sentences = []
for fname in tqdm(list(os.listdir('data/ami_public_manual_1.6.2/words/'))):
    if not fname.endswith('.xml'):
        continue
    sentences += extract_sentences(fname)

  0%|          | 0/687 [00:00<?, ?it/s]

In [119]:
sentence_df = pd.DataFrame(sentences, columns=['starttime', 'text', 'fname'])

In [120]:
len(sentence_df)

83436

In [121]:
sentence_df.to_csv('data/ami.sentence.tsv', index=False, sep='\t')